In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Fri Jan 26 18:37:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
# Pip install necessary package
!pip install --upgrade --quiet  pgvector
!pip install --upgrade --quiet  langchain-openai
!pip install --upgrade --quiet  psycopg2-binary
!pip install --upgrade --quiet  tiktoken

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.20 requires torch==2.0.1, but you have torch 2.1.0 wh

In [ ]:
pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.pgvector import PGVector
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'langchain_community'

## Data

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

load the existing vector db from disk

In [ ]:
CONNECTION_STRING = "***************"

store = PGVector(
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
    collection_name="Physicianai",

)

## Llama 2 13B

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. give answer for any questions.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

> Indented block



In [ ]:
result = qa_chain("im facing stomack pain after eat rice what is the reason?. if question is not related to provided data don't give answer from this provided data's")

 Based on the given information, it is not possible to determine the exact cause of stomach pain after eating rice. However, there are several potential reasons why someone might experience stomach pain after consuming rice or any other food item. Some possible causes include:

1. Food allergies or intolerances: Some people may be allergic or intolerant to certain ingredients in rice, such as gluten or lactose, which can cause digestive issues and stomach pain.
2. Gastrointestinal issues: Conditions like irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), and gastroesophageal reflux disease (GERD) can cause stomach pain and discomfort after eating certain foods, including rice.
3. Infection or virus: Stomach pain can be a symptom of a bacterial or viral infection, such as food poisoning or a stomach flu.
4. Dietary habits: Consuming large amounts of rice or other high-carbohydrate foods can lead to indigestion, bloating, and stomach pain in some individuals.
5. Other medi

In [ ]:
len(result["source_documents"])

2

In [ ]:
print(result["source_documents"][0].page_content)

Meta Reports Second  Quarter 2023  Results
MENLO PARK, Calif. – July 26, 2023  – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the 
quarter ended June 30, 2023 .
"We had a good quarter. We continue to see strong engagement across our apps and we have the most exciting roadmap I've 
seen in a while with Llama 2, Threads, Reels, new AI products in the pipeline, and the launch of Quest 3 this fall," said Mark 
Zuckerberg, Meta founder and CEO.
Second Quarter 2023  Financial Highlights
Three Months Ended June 30,
In millions, except percentages and per share amounts 2023 2022 % Change
Revenue $ 31,999 $ 28,822 11%
Costs and expenses  22,607  20,464 10%
Income from operations $ 9,392 $ 8,358 12%
Operating margin  29 %  29 %
Provision for income taxes $ 1,505 $ 1,499 —%
Effective tax rate  16 %  18 %
Net income $ 7,788 $ 6,687 16%
Diluted earnings per share (EPS) $ 2.98 $ 2.46 21%
Second Quarter 2023  Operational and Other Financial Highlights


In [ ]:
result = qa_chain("What is the per share revenue for Tesla during 2023?")

 Based on the information provided in the financial statements, the per share revenue for Tesla during 2023 can be calculated as follows:

Total Revenue for 2023: $48,256 (from the Consolidated Statements of Operations)

Number of shares outstanding: 249,270 (from the Balance Sheet)

Per Share Revenue for 2023: $48,256 / 249,270 = $195.62

Therefore, the per share revenue for Tesla during 2023 was $195.62.


In [ ]:
result = qa_chain("What is the per share revenue for Nvidia during 2023?")

 Based on the information provided in the financial statements, the per share revenue for Nvidia during 2023 was $0.83. This is calculated by dividing the total revenue for the period ($7,192 million) by the weighted average number of shares outstanding during the period (2,470 million).


In [ ]:
print(result["source_documents"][1].page_content)

PART I. FINANCIAL INFORMATION
ITEM 1.  FINANCIAL STATEMENTS (UNAUDITED)
NVIDIA CORPORATION AND SUBSIDIARIES
CONDENSED CONSOLIDATED STATEMENTS OF INCOME
(In millions, except per share data)
(Unaudited)
 Three Months Ended
 April 30, May 1,
2023 2022
Revenue $ 7,192 $ 8,288 
Cost of revenue 2,544 2,857 
Gross profit 4,648 5,431 
Operating expenses   
Research and development 1,875 1,618 
Sales, general and administrative 633 592 
Acquisition termination cost — 1,353 
Total operating expenses 2,508 3,563 
Income from operations 2,140 1,868 
Interest income 150 18 
Interest expense (66) (68)
Other, net (15) (13)
Other income (expense), net 69 (63)
Income before income tax 2,209 1,805 
Income tax expense 166 187 
Net income $ 2,043 $ 1,618 
Net income per share:
Basic $ 0.83 $ 0.65 
Diluted $ 0.82 $ 0.64 
Weighted average shares used in per share computation:
Basic 2,470 2,506 
Diluted 2,490 2,537 
See accompanying Notes to Condensed Consolidated Financial Statements.
3


In [ ]:
result = qa_chain("What is the estimated YOY revenue for Meta during 2023?")

 Based on the information provided in the press release, Meta's estimated YOY revenue for 2023 is expected to be between $32-34.5 billion, which represents an increase of 11-16% over 2022.


In [ ]:
result = qa_chain("What is the estimated YOY revenue for Tesla during 2023?")

 Based on the information provided in the quarterly report, the estimated year-over-year (YOY) revenue for Tesla during 2023 is not explicitly stated. However, we can calculate an estimate based on the information provided.

For the six months ended June 30, 2023, Tesla's revenues were $41,231 million, representing a decrease of $768 million compared to the same period in 2022. To estimate the YOY revenue for the full year 2023, we can assume that the second half of 2023 will be similar to the first half in terms of revenue growth.

Assuming no significant changes in market conditions or business operations, we can project the full-year 2023 revenue based on the first half's performance as follows:

Full-year 2023 revenue = (First half 2023 revenue x 2) + (Second half 2022 revenue x 1)

where "Second half 2022 revenue" represents the revenue generated during the last six months of 2022.

Based on the information provided, the second half 2022 revenue was $35,690 million. Therefore, the

In [ ]:
result = qa_chain("What is the estimated YOY revenue for Nvidia during 2023?")

 Based on the information provided, I cannot estimate the exact year-over-year (YOY) revenue for Nvidia during 2023. The condensed consolidated financial statements only provide information up to the first quarter of fiscal year 2024, which ended on April 30, 2023. There are three more quarters left in fiscal year 2023, and we do not have information on the remaining quarters. Therefore, I cannot estimate the full-year YOY revenue for Nvidia during 2023.


In [ ]:
result = qa_chain(
    "Which company is more profitable during 2023 Meta, Nvidia or Tesla and why?"
)

 I cannot determine which company is more profitable during 2023 between Meta, Nvidia, and Tesla based on the information provided. The passage only discusses Nvidia's financial condition and results of operations, but does not provide any information about Meta or Tesla's financial performance during 2023. Additionally, the passage notes that investors should carefully consider the risk factors set forth in Nvidia's filings with the SEC before making any investment decisions, which suggests that the profitability of these companies may be subject to various factors and uncertainties. Therefore, without access to the financial information of all three companies, it is impossible to accurately compare their profitability during 2023.


In [ ]:
result = qa_chain(
    "Choose one company to invest (Tesla, Nvidia or Meta) to maximize your profits for the long term (10+ years)?"
)

 I cannot provide a definitive answer to this question, as it is not appropriate for me to give financial advice or make recommendations on specific investments. However, based on the information provided in the context, NVIDIA appears to be a strong company with a diverse range of businesses and a history of innovation, including the development of GPU architecture for various applications such as scientific computing, AI, data science, and more. Additionally, the company has a solid track record of growth and profitability, and has been successful in transitioning to new products and business models over time. These factors could potentially contribute to long-term success and profitability for investors. However, it is important to conduct thorough research and consider multiple factors before making any investment decisions.
